<a href="https://colab.research.google.com/github/ArtemDorofeev/Sequence_ganlytic/blob/main/GA_sequences_transactions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Импорт библиотек

import pandas as pd
import numpy as np
import re

In [ ]:
# Авторизируемся

from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
# Формируем запрос к ga_sessions за период с 01.07.2017 по 31.07.2017 

%%bigquery dataset --project tactical-coder-376517

SELECT visitId, fullVisitorId, date, visitNumber, totals.transactions, trafficSource, hits
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
  _TABLE_SUFFIX BETWEEN '20170701'
  AND '20170731'

Query is running:   0%|          |

Downloading:   0%|          |

## Обработка данных

In [ ]:
# Сохраним подгруженные данные в отдельный датафрейм

df = dataset

In [ ]:
# Размер датасета и количество транзакций

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71812 entries, 0 to 71811
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   visitId        71812 non-null  Int64 
 1   fullVisitorId  71812 non-null  object
 2   date           71812 non-null  object
 3   visitNumber    71812 non-null  Int64 
 4   transactions   1031 non-null   Int64 
 5   trafficSource  71812 non-null  object
 6   hits           71812 non-null  object
dtypes: Int64(3), object(4)
memory usage: 4.0+ MB


In [ ]:
# Шапка данных

df.head(5)

,visitId,fullVisitorId,date,visitNumber,transactions,trafficSource,hits
0,1499117293,1018154947831642966,20170703,1,<NA>,"{'referralPath': None, 'campaign': '(not set)'...","[{'hitNumber': 1, 'time': 0, 'hour': 14, 'minu..."
1,1499132474,1770622829105680991,20170703,1,<NA>,"{'referralPath': None, 'campaign': '(not set)'...","[{'hitNumber': 1, 'time': 0, 'hour': 18, 'minu..."
2,1499091579,9063910466778307075,20170703,1,<NA>,"{'referralPath': None, 'campaign': '(not set)'...","[{'hitNumber': 1, 'time': 0, 'hour': 7, 'minut..."
3,1499120952,5760108313091568580,20170703,1,<NA>,"{'referralPath': None, 'campaign': '(not set)'...","[{'hitNumber': 1, 'time': 0, 'hour': 15, 'minu..."
4,1499101872,1793860145379975455,20170703,1,<NA>,"{'referralPath': None, 'campaign': '(not set)'...","[{'hitNumber': 1, 'time': 0, 'hour': 10, 'minu..."


In [ ]:
# Вытаскиваем список id транзакций из массива "hits", если они есть

def transaction_id(row):
    lst = []
    for i in row:
        try:
            if i['transaction']:
              if i['transaction']['transactionId']:
                n = i['transaction']['transactionId']
                lst.append(n)                
        except:
            lst = lst
    return lst

In [ ]:
df['trans_id'] = df['hits'].apply(transaction_id)

In [ ]:
# Выделяем source и medium и объединяем в одно значение

df['source'] = df['trafficSource'].apply(lambda x: x['source'][1:-1] if x['source'][0] == '(' else x['source'])
df['medium'] = df['trafficSource'].apply(lambda x: x['medium'][1:-1] if x['medium'][0] == '(' else x['medium'])
df['origin'] = df['source'] + "_" + df['medium']

In [ ]:
# Замение отсутствующие значения транзакций на ноль

df.loc[df['transactions'].isna(), 'transactions'] = 0

In [ ]:
df = df[['visitId', 'fullVisitorId', 'date', 'visitNumber', 'transactions', 'trans_id', 'source', 'medium', 'origin']]
df.sample(10)

,visitId,fullVisitorId,date,visitNumber,transactions,trans_id,source,medium,origin
67518,1500482850,928775034401331,20170719,1,0,[],google,organic,google_organic
68925,1500482824,4133598777766471758,20170719,1,0,[],direct,none,direct_none
54316,1500441871,4913843427593856760,20170718,1,0,[],google,cpc,google_cpc
830,1499090997,0845720339271024704,20170703,1,0,[],google,organic,google_organic
9586,1500938140,260696920918575699,20170724,1,0,[],google,organic,google_organic
28170,1498987686,2912791699033852244,20170702,2,0,[],m.facebook.com,referral,m.facebook.com_referral
53041,1501280946,9605304896517637720,20170728,2,0,[],google,organic,google_organic
62909,1499279825,3231137981243511563,20170705,1,0,[],youtube.com,referral,youtube.com_referral
31723,1500311886,0128830210468179660,20170717,4,1,"[ORD201707172477, ORD201707172477]",google,organic,google_organic
41164,1501160905,0369551715074056796,20170727,1,0,[],direct,none,direct_none


In [ ]:
# Сформируем датасет из уникальных транзакционных визитов

df1 = df[df['trans_id'].astype(bool)]
duplicate = df1['fullVisitorId'].duplicated()
df1 = df1.drop(index=df1[duplicate].index, axis=0)

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 964 entries, 22 to 70358
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   visitId        964 non-null    Int64 
 1   fullVisitorId  964 non-null    object
 2   date           964 non-null    object
 3   visitNumber    964 non-null    Int64 
 4   transactions   964 non-null    Int64 
 5   trans_id       964 non-null    object
 6   source         964 non-null    object
 7   medium         964 non-null    object
 8   origin         964 non-null    object
dtypes: Int64(3), object(6)
memory usage: 78.1+ KB


In [ ]:
df1['origin'].value_counts()

direct_none                     605
google_organic                  274
google_cpc                       44
google_cpm                       15
sites.google.com_referral         7
dfa_cpm                           4
Partners_affiliate                4
yahoo_organic                     3
bing_organic                      2
mail.google.com_referral          2
m.facebook.com_referral           1
calendar.google.com_referral      1
outlook.live.com_referral         1
youtube.com_referral              1
Name: origin, dtype: int64

In [ ]:
# Создаем функцию по генерации списка последовательных кортежей значений: 
# - номер визита (+сортировка по возрастанию)
# - источник визита
# - количество транзакций

def form_range(row):
  x = df.loc[df['fullVisitorId'] == row]
  n = x['visitNumber'].values
  ts = x['origin'].values
  tr = x['transactions'].values
  dic = {n[i]: [ts[i], tr[i]] for i in range(len(n))}
  val = sorted(dic.items())
  return val

In [ ]:
df1['list'] = df1['fullVisitorId'].apply(form_range)

In [ ]:
df1.sample(8)

,visitId,fullVisitorId,date,visitNumber,transactions,trans_id,source,medium,origin,list
39105,1499712836,5812572929202367937,20170710,1,1,"[ORD201707102699, ORD201707102699]",google,organic,google_organic,"[(1, [google_organic, 1]), (2, [google_organic..."
7653,1501468003,15618322761092379,20170730,4,1,"[ORD201707301717, ORD201707301717]",direct,none,direct_none,"[(4, [direct_none, 1])]"
36138,1501504847,5258378690008715588,20170731,1,1,"[ORD201707312271, ORD201707312271]",direct,none,direct_none,"[(1, [direct_none, 1]), (2, [direct_none, 0])]"
12846,1500643442,0953889584295924240,20170721,2,1,"[ORD201707212377, ORD201707212377]",direct,none,direct_none,"[(2, [direct_none, 1])]"
59230,1500993100,6052206455243513477,20170725,4,1,"[ORD201707252566, ORD201707252566]",direct,none,direct_none,"[(1, [direct_none, 0]), (2, [direct_none, 0]),..."
66555,1500860215,5952422370450576468,20170723,2,1,"[ORD201707231885, ORD201707231885]",direct,none,direct_none,"[(2, [direct_none, 1])]"
60441,1501013364,1424355572088327868,20170725,3,1,"[ORD201707252308, ORD201707252308]",google,organic,google_organic,"[(1, [google_organic, 0]), (2, [google_cpc, 0]..."
7528,1501459755,713731982901764318,20170730,15,1,"[ORD201707301542, ORD201707301542]",direct,none,direct_none,"[(10, [direct_none, 1]), (11, [direct_none, 0]..."


In [ ]:
# Оставляем цепочку визитов от начального до момента совершения первой транзакции

def creat_way(row):
  for i in range(len(row)):
    if row[i][1][1] != 0:
      n = i
      val = row[:i+1]
      break
    else:
      val = []
  return val   

In [ ]:
df1['way'] = df1['list'].apply(creat_way)

## Создаем таблицы

### 1) Первый вариант: 
Создаем таблицы в которые входят все цепочки визитов до первой транзакции, вошедшие в выбранный период. В т.ч. начинающиеся не с 1-го визита пользователя

In [ ]:
df_all = df1[['visitId', 'fullVisitorId', 'date', 'visitNumber', 'way']]
df_all.sample(8)

,visitId,fullVisitorId,date,visitNumber,way
53775,1500420814,1774472718303082739,20170718,3,"[(1, [google_organic, 0]), (2, [google_organic..."
18661,1499605874,1693439860882386132,20170709,1,"[(1, [google_organic, 1])]"
53564,1501265091,9521438372294298977,20170728,3,"[(2, [google_organic, 0]), (3, [google_cpc, 1])]"
20654,1499799777,5224330515080725566,20170711,1,"[(1, [direct_none, 1])]"
48097,1499455839,7447137895250670218,20170707,2,"[(1, [direct_none, 0]), (2, [direct_none, 1])]"
59023,1499984473,2966465528641096132,20170713,1,"[(1, [direct_none, 1])]"
12840,1500650071,4045831322438555082,20170721,1,"[(1, [google_organic, 1])]"
15209,1501361792,1393633173328801040,20170729,3,"[(1, [direct_none, 0]), (2, [direct_none, 0]),..."


In [ ]:
# Объединим значения источника трафика в строковое значение последовательности

def way_transaction(row):
  lst = []
  for i in row:
    lst.append(i[1][0])
  w = (' ->> ').join(lst)
  return w

In [ ]:
df_all['way_to_transaction'] = df_all['way'].apply(way_transaction)

<ipython-input-165-8bc23345cc3b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all['way_to_transaction'] = df_all['way'].apply(way_transaction)


In [ ]:
df_all = df_all[['visitId', 'fullVisitorId', 'date', 'visitNumber', 'way_to_transaction']]
df_all.sample(8)

,visitId,fullVisitorId,date,visitNumber,way_to_transaction
12745,1499377019,623238387362841295,20170706,1,direct_none
59015,1499961383,5807278314451544984,20170713,2,direct_none ->> direct_none
12849,1500681737,3117505679893009709,20170721,3,direct_none ->> direct_none ->> direct_none
12731,1499384037,3680806121011427493,20170706,2,direct_none ->> direct_none
48101,1499468129,051516119104547275,20170707,1,google_organic
31837,1500307465,039400440658825933,20170717,3,direct_none ->> direct_none ->> direct_none
58936,1499974599,3550517741801560701,20170713,5,google_cpc
60542,1501027632,9654518257837426461,20170725,4,google_cpm ->> direct_none ->> direct_none ->>...


In [ ]:
df_all['way_to_transaction'].value_counts()[:10]

direct_none                                                                    333
google_organic                                                                 154
direct_none ->> direct_none                                                    136
direct_none ->> direct_none ->> direct_none                                     53
google_organic ->> google_organic                                               44
direct_none ->> direct_none ->> direct_none ->> direct_none                     30
google_cpc                                                                      29
google_organic ->> google_organic ->> google_organic                            15
direct_none ->> direct_none ->> direct_none ->> direct_none ->> direct_none     13
google_cpc ->> google_cpc                                                        9
Name: way_to_transaction, dtype: int64

### 2) Второй вариант: 
Создаем таблицы в которые входят только цепочки визитов до первой транзакции, начинающиеся с 1-го визита пользователя. (не включая хвост предыдущего периода)

In [ ]:
df_full = df1[['visitId', 'fullVisitorId', 'date', 'visitNumber', 'way']]
df_full.sample(8)

,visitId,fullVisitorId,date,visitNumber,way
42797,1499879049,6979205517954044794,20170712,3,"[(1, [google_cpm, 0]), (2, [google_cpm, 0]), (..."
20560,1499798142,7782965489086957079,20170711,6,"[(2, [direct_none, 0]), (3, [direct_none, 0]),..."
7705,1500904908,4871978976437661341,20170724,1,"[(1, [google_organic, 1])]"
59229,1500993067,7115815619589342932,20170725,3,"[(1, [direct_none, 0]), (2, [direct_none, 0]),..."
7745,1500896954,7285581814755922222,20170724,1,"[(1, [direct_none, 1])]"
30939,1500052337,6800222720402902812,20170714,1,"[(1, [direct_none, 1])]"
10065,1500926348,2888994662401288163,20170724,1,"[(1, [google_organic, 1])]"
48230,1499449007,4068416534578096540,20170707,5,"[(1, [direct_none, 0]), (2, [direct_none, 0]),..."


In [ ]:
# Выделяем цепочки начинающиеся с визита №1

def way_full(row):
  try:
    if row[0][0] == 1:
      x = 'full'
    else:
      x = 'tail'
  except:
    x = 'empty'
  return x

In [ ]:
df_full['equal'] = df_full['way'].apply(way_full)

<ipython-input-172-6932ca3b882b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_full['equal'] = df_full['way'].apply(way_full)


In [ ]:
df_full = df_full.loc[df_full['equal'] == 'full']
df_full.sample(8)

,visitId,fullVisitorId,date,visitNumber,way,equal
53681,1500405332,7824759881998320555,20170718,1,"[(1, [google_organic, 1])]",full
16757,1501390392,9972641923092309898,20170729,2,"[(1, [google_organic, 0]), (2, [google_organic...",full
69126,1500521313,0668741043882663600,20170719,2,"[(1, [google_cpc, 0]), (2, [google_cpc, 1])]",full
53689,1500398842,9868557064314705324,20170718,2,"[(1, [google_organic, 0]), (2, [google_organic...",full
69066,1500481324,8478498564954638951,20170719,2,"[(1, [direct_none, 0]), (2, [direct_none, 1])]",full
7684,1500952565,4073209630928256180,20170724,2,"[(1, [google_organic, 0]), (2, [google_organic...",full
20622,1499793400,7213484590531142958,20170711,1,"[(1, [direct_none, 1])]",full
70201,1500591500,7089296120636038584,20170720,3,"[(1, [google_organic, 0]), (2, [yahoo_organic,...",full


In [ ]:
df_full['way_to_transaction'] = df_full['way'].apply(way_transaction)

In [ ]:
df_full = df_full[['visitId', 'fullVisitorId', 'date', 'visitNumber', 'way_to_transaction']]
df_full.sample(8)

,visitId,fullVisitorId,date,visitNumber,way_to_transaction
31085,1500062238,4737716055104076300,20170714,1,direct_none
39214,1501197828,3563510904717248245,20170727,5,google_organic ->> google_organic ->> google_o...
44387,1501077990,5248418277725722133,20170726,1,google_organic
39252,1501167382,449424538549488734,20170727,1,direct_none
70220,1500583858,1863054289029276927,20170720,6,direct_none ->> direct_none ->> direct_none ->...
42964,1499880557,6005829572969771144,20170712,3,direct_none ->> google_organic ->> google_organic
39726,1501171464,6644273643966013405,20170727,2,direct_none ->> direct_none
12840,1500650071,4045831322438555082,20170721,1,google_organic


In [ ]:
df_full['way_to_transaction'].value_counts()[:10]

direct_none                                                    217
google_organic                                                 126
direct_none ->> direct_none                                     84
direct_none ->> direct_none ->> direct_none                     34
google_organic ->> google_organic                               32
google_cpc                                                      20
direct_none ->> direct_none ->> direct_none ->> direct_none     19
google_organic ->> google_organic ->> google_organic            11
google_cpc ->> google_cpc                                        8
google_cpm                                                       8
Name: way_to_transaction, dtype: int64

для сравнения

In [ ]:
df_all['way_to_transaction'].value_counts()[:10]

direct_none                                                                    333
google_organic                                                                 154
direct_none ->> direct_none                                                    136
direct_none ->> direct_none ->> direct_none                                     53
google_organic ->> google_organic                                               44
direct_none ->> direct_none ->> direct_none ->> direct_none                     30
google_cpc                                                                      29
google_organic ->> google_organic ->> google_organic                            15
direct_none ->> direct_none ->> direct_none ->> direct_none ->> direct_none     13
google_cpc ->> google_cpc                                                        9
Name: way_to_transaction, dtype: int64